## Tensorflow Serving Version 1.13 Example

Train and export a simple Softmax Regression TensorFlow model.
The model is from the TensorFlow "MNIST For ML Beginner" tutorial. This program
simply follows all its training instructions, and uses TensorFlow SavedModel to
export the trained model with proper signatures that can be loaded by standard
tensorflow_model_server.
<br>
Usage: mnist_saved_model.py [--training_iteration=x] [--model_version=y] export_dir

In [1]:
from __future__ import print_function

import os
import sys
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.contrib.session_bundle import exporter

In [2]:
tf.app.flags.DEFINE_integer('training_iteration', 1000, 'number of training iterations.')
tf.app.flags.DEFINE_integer('model_version', 1, 'version number of the model.')
tf.app.flags.DEFINE_string('export_path_base', './model', 'Model directory.')
FLAGS = tf.app.flags.FLAGS

In [3]:
train_x = np.array([1.60, 1.62, 1.68, 1.69, 1.70, 1.70, 1.71, 1.72, 1.80, 1.76, 1.77, 1.78, 1.77, 1.78, 1.79, 1.80, 1.81, 1.82, 1.83, 1.94]).reshape(-1, 1)
train_y = np.array([60.5, 58.8, 62.7, 62.8, 63.1, 58.9, 61.5, 65.8, 70.0, 65.8, 66.8, 68.0, 66.7, 60.8, 72.4, 70.5, 72.8, 75.8, 74.6, 83.2]).reshape(-1, 1)

In [4]:
def main(_):
    x = tf.placeholder('float', shape=[None, 1], name = 'x')
    y_ = tf.placeholder('float', shape=[None, 1], name = 'y')
    w = tf.get_variable(name = 'w', shape = [1,1], initializer = tf.truncated_normal_initializer)
    b = tf.get_variable(name = 'b', shape = [1], initializer = tf.zeros_initializer)
    y = tf.matmul(x, w) + b
    ms_loss = tf.reduce_mean((y - y_)**2)
    init = tf.global_variables_initializer()
    train_step = tf.train.GradientDescentOptimizer(0.005).minimize(ms_loss)
    train_loss = []
    with tf.Session() as sess:
        sess.run(init)
        for it in range(FLAGS.training_iteration):
            loss, _ = sess.run([ms_loss, train_step], feed_dict={x: train_x, y_: train_y})
            train_loss.append(loss)

        print('Training error %g' % loss)

        export_path = os.path.join(
            tf.compat.as_bytes(FLAGS.export_path_base),
            tf.compat.as_bytes(str(FLAGS.model_version))
        )

        print('Exporting trained model to', export_path)
        builder = tf.saved_model.builder.SavedModelBuilder(export_path)

        tensor_info_x = tf.saved_model.utils.build_tensor_info(x)
        tensor_info_y = tf.saved_model.utils.build_tensor_info(y)

        prediction_signature = (
            tf.saved_model.signature_def_utils.build_signature_def(
                inputs={'input': tensor_info_x},
                outputs={'output': tensor_info_y},
                method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
            )
        )

        legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')

        builder.add_meta_graph_and_variables(
            sess, [tf.saved_model.tag_constants.SERVING],
            signature_def_map={
                'prediction':
                prediction_signature,
            },
            legacy_init_op=legacy_init_op
        )

        builder.save()
        print('Done exporting!')

In [5]:
if __name__ == '__main__':
    tf.app.run()

Instructions for updating:
Colocations handled automatically by placer.
Training error 18.9956
Exporting trained model to b'./model/1'
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./model/1/saved_model.pb
Done exporting!


SystemExit: 

/home/bai/.virtualenvs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### Start Tensorflow Serving (don't specify model version)

docker run -p 8501:8501 --mount type=bind,source=/home/bai/PycharmProjects/tensorflow_for_enginner/11_tf_serving/model,target=/models/my_model -e MODEL_NAME=my_model -t tensorflow/serving

curl -X POST http://localhost:8501/v1/models/my_model:predict -d '{"signature_name":"prediction","instances":[{"input":[1.8]}]}'